In [2]:
import json
import os
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch
import numpy as np
from tqdm.auto import tqdm


In [22]:
train_dir = '/datasets/inat_comp/2021/train_mini'
val_dir = '/datasets/inat_comp/2021/val'
train_json = '/datasets/inat_comp/2021/train_mini.json'
val_json = '/datasets/inat_comp/2021/val.json'
source = '/datasets/inat_comp/2021/'
test_json = '/datasets/inat_comp/2021/public_test.json'
test = '/datasets/inat_comp/2021/public_test'

In [23]:
import os

print(os.listdir(source)) # here '.' for current directory.

['train.json', 'train_mini.json.tar.gz', 'train.tar.gz', 'public_test.json', 'train_mini.json', 'train', 'val.json', 'val.tar.gz', 'train.json.tar.gz', 'public_test', 'train_mini', 'val', 'val.json.tar.gz', 'public_test.tar.gz', 'public_test.json.tar.gz', 'train_mini.tar.gz']


In [25]:
def load(js
with open(test_json, 'r') as f:
    test_json = json.load(f)

In [27]:
with open(val_json, 'r') as f:
    val_json = json.load(f)

In [28]:
val_json

{'info': {'year': 2021,
  'verion': 1,
  'description': 'iNaturalist Species Classification Dataset Testing Split.',
  'contributor': 'Grant Van Horn and the Visipedia Team.',
  'url': 'https://github.com/visipedia/inat_comp',
  'date_created': '2021-03-01 12:34:38'},
 'images': [{'id': 2686843,
   'width': 284,
   'height': 222,
   'file_name': 'val/03938_Animalia_Chordata_Aves_Passeriformes_Meliphagidae_Ptilotula_penicillata/df8edd4c-fbb4-4886-8600-a429e5efac23.jpg',
   'license': 2,
   'rights_holder': 'megatherium',
   'date': '2007-10-31 00:00:00+00:00',
   'latitude': -21.93073,
   'longitude': 114.12239,
   'location_uncertainty': None},
  {'id': 2686844,
   'width': 500,
   'height': 375,
   'file_name': 'val/03583_Animalia_Chordata_Aves_Cuculiformes_Cuculidae_Coccyzus_erythropthalmus/fc35080c-5ace-4485-a21f-b1447f27efc7.jpg',
   'license': 1,
   'rights_holder': 'rpayne',
   'date': '2011-07-15 00:00:00+00:00',
   'latitude': 44.02901,
   'longitude': -73.17711,
   'location_u

In [26]:
test_json

{'info': {'year': 2021,
  'verion': 1,
  'description': 'iNaturalist Species Classification Dataset Testing Split.',
  'contributor': 'Grant Van Horn and the Visipedia Team.',
  'url': 'https://github.com/visipedia/inat_comp',
  'date_created': '2021-03-01 12:34:38'},
 'images': [{'id': 1,
   'width': 470,
   'height': 500,
   'file_name': 'public_test/1.jpg',
   'license': 1,
   'rights_holder': 'risonkj',
   'date': '2019-10-27 03:09:00+00:00',
   'latitude': 10.29758,
   'longitude': 76.2566,
   'location_uncertainty': 2187},
  {'id': 2,
   'width': 375,
   'height': 500,
   'file_name': 'public_test/2.jpg',
   'license': 1,
   'rights_holder': 'Jakob Ingle',
   'date': '2020-05-08 19:16:24+00:00',
   'latitude': 34.56587,
   'longitude': -118.13512,
   'location_uncertainty': 5},
  {'id': 3,
   'width': 500,
   'height': 500,
   'file_name': 'public_test/3.jpg',
   'license': 1,
   'rights_holder': 'Erin',
   'date': '2020-07-21 19:00:00+00:00',
   'latitude': -12.66681,
   'longit

In [4]:
import open_clip

# Load the BioCLIP model and its preprocessors
model, _, preprocess = open_clip.create_model_and_transforms('hf-hub:imageomics/bioclip')
model.eval()  # Ensure model is in evaluation mode = open_clip.create_model_and_transforms('hf-hub:imageomics/bioclip')
tokenizer = open_clip.get_tokenizer('hf-hub:imageomics/bioclip')
device = "cuda" if torch.cuda.is_available() else "cpu"


In [5]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [12]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

In [6]:
#label_tokens = tokenizer(clip_labels, return_tensors='pt', padding=True)
model.to("cuda")

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine

In [ ]:
# from torchvision.transforms import Resize

# # Resize images before preprocessing
# resize = Resize((256, 256))  # Adjust size as necessary

# # Modify image loading and preprocessing
# images = [resize(Image.open(image_paths[j])) for j in range(i, i_end)]
# image_tensors = torch.stack([preprocess(image).unsqueeze(0) for image in images]).to(model.device)


In [7]:
image_dir = '/scratch/ssd004/scratch/junejory/val2019'
image_paths = []
image_labels = []


for annotation in annotations['annotations']:
    image_info = next(image for image in annotations['images'] if image['id'] == annotation['image_id'])
    image_path = os.path.join(image_dir, image_info['file_name'])
    image_paths.append(image_path)
    image_labels.append(annotation['category_id'])

In [12]:
image_labels

[644,
 597,
 883,
 300,
 881,
 771,
 607,
 698,
 714,
 316,
 726,
 27,
 640,
 756,
 73,
 594,
 179,
 440,
 859,
 5,
 401,
 928,
 216,
 200,
 983,
 350,
 223,
 438,
 834,
 248,
 127,
 451,
 828,
 893,
 456,
 811,
 272,
 627,
 291,
 960,
 583,
 217,
 17,
 637,
 99,
 703,
 18,
 458,
 1000,
 499,
 704,
 276,
 611,
 719,
 584,
 1005,
 511,
 886,
 938,
 370,
 482,
 79,
 367,
 254,
 605,
 860,
 382,
 384,
 420,
 821,
 548,
 944,
 484,
 41,
 963,
 574,
 320,
 144,
 786,
 549,
 981,
 47,
 434,
 194,
 629,
 119,
 812,
 105,
 431,
 282,
 759,
 295,
 83,
 469,
 59,
 499,
 965,
 496,
 364,
 309,
 740,
 412,
 53,
 915,
 882,
 585,
 167,
 82,
 772,
 35,
 268,
 776,
 855,
 668,
 43,
 16,
 171,
 114,
 166,
 57,
 786,
 977,
 301,
 503,
 858,
 690,
 385,
 985,
 11,
 637,
 72,
 305,
 327,
 465,
 551,
 657,
 418,
 644,
 533,
 104,
 442,
 389,
 413,
 734,
 123,
 303,
 680,
 617,
 328,
 51,
 419,
 514,
 621,
 290,
 466,
 110,
 397,
 697,
 426,
 323,
 509,
 892,
 341,
 140,
 313,
 79,
 175,
 850,
 378,
 870,


In [45]:
tokenized_labels = tokenizer(clip_labels).to(device)

# Ensure tokenized labels are in the right format
# Note: This assumes the tokenizer returns a tensor of shape [num_labels, max_seq_length]
if tokenized_labels.ndimension() == 1:
    tokenized_labels = tokenized_labels.unsqueeze(0)

# Get the label embeddings
with torch.no_grad():
    label_emb = model.encode_text(tokenized_labels)

# label_embeddings now contains the embeddings for your labels
label_emb /= label_emb.norm(dim=-1, keepdim=True)

print(label_emb.shape) 

torch.Size([1010, 512])


In [22]:
for image_path, label_id in zip(image_paths, image_labels):
    # Load and preprocess image
    image = preprocess(Image.open(image_path)).unsqueeze(0)

In [46]:
preds = []
batch_size = 32

for i in tqdm(range(0, len(image_paths), batch_size)):
    i_end = min(i + batch_size, len(image_paths))
    images = [Image.open(image_paths[j]) for j in range(i, i_end)]
    image_tensors = torch.stack([preprocess(image) for image in images]).to(device)

    with torch.no_grad(), torch.cuda.amp.autocast():
        img_emb = model.encode_image(image_tensors)
        img_emb /= img_emb.norm(dim=-1, keepdim=True)
        scores = (100.0 * img_emb @ label_emb.T).softmax(dim=-1)

    batch_preds = torch.argmax(scores, dim=1).cpu().numpy()
    preds.extend(batch_preds)

# Calculate accuracy
correct = sum(1 for pred, label in zip(preds, image_labels) if pred == label)
accuracy = correct / len(image_labels)
print(f"Accuracy: {accuracy:.4f}")

  0%|          | 0/95 [00:00<?, ?it/s]

Accuracy: 0.7063
